In [134]:
import pandas as pd
from datetime import date

df_base_month = pd.read_csv("japan_import (1).csv", on_bad_lines='skip')
df_base_context = pd.read_csv("japan_import (1).csv", header=6)
df_iso = pd.read_csv("JP_ISO_index.csv")

# create a function to add new column call "Date" and add corresponding date
def add_date_column(df_month, df_context):
    for index, values in df_month.iterrows():
        if str(df_month.loc[index, "Commodity by Country"]).split("/")[0][1:3] == "20":
            year = str(df_month.loc[index, "Commodity by Country"]).split("/")[0][1:]
            month = str(df_month.loc[index, "Commodity by Country"]).split("/")[1]
            if len(month) == 1:
                month = '0' + str(month)
            
    df_context["DATE"] = year + '-' + month + '-' + '01'
    return df_context

def create_empty_dataframe():
    df_template = pd.DataFrame(columns=["DATE", 
                                        "HS_CODE", 
                                        "ISO", 
                                        "QUANTITY_PCS", 
                                        "VALUE_USD"])
    return df_template                                    
            
# def add_to_new_df(data_df, base_df):
#     for index, value in data_df.iterrows():

# add ISO code to dataframe using left join on country code column
def update_iso(df,df_iso, country_col):
    df[country_col] = df[country_col].str.replace("'", "").astype(int)
# add country for import file iso code from iso csv file 
    df = df.merge(df_iso[['Country_Code', 'ISO']], left_on=country_col, right_on='Country_Code', how='left')
    return df
 
# define function to change HS code column name and remove symbols in the rows 
# remember to have amount1 unit1 amount 2 unit2 order.    
def update_hs_code(df,col_name):
    df[col_name] = df[col_name].str.replace(".","")
    df[col_name] = df[col_name].str.replace("-000","").astype(int)
    df.rename(columns={col_name : "HS_CODE"}, inplace=True)
    return df

# define function to remove symbols in the chosen columns
# note the empty column values will be added as integer 0
# Since most of the countries does not have amount, to be uniform
# Japan amount will be removed
### Forex exchange rate link https://www.ofx.com/en-us/forex-news/historical-exchange-rates/usd/hkd/
def clean_columns(df, col_1, col_2, col_3, col_4, col_5):
    df["QUANTITY_PCS"] = 0
    for index, value in df.iterrows():
       if df.loc[index, col_1] == 'NO':
           if df.loc[index, col_3] == None:
               df.loc[index, "QUANTITY_PCS"] = 0
           elif df.loc[index, col_3] == '-':
               df.loc[index, "QUANTITY_PCS"] = 0  
           else:          
               df.loc[index, "QUANTITY_PCS"] = df.loc[index, col_3]
               
       elif df.loc[index, col_2] == 'NO':
           if df.loc[index, col_4] == None:
               df.loc[index, "QUANTITY_PCS"] = 0
           elif df.loc[index, col_4] == '-':
               df.loc[index, "QUANTITY_PCS"] = 0  
           else:          
               df.loc[index, "QUANTITY_PCS"] = df.loc[index, col_4]
    df[col_5] = df[col_5].replace("-",0).astype(float)
    df[col_5] = (df[col_5] * 1000 * 0.008926).astype(int)
    df.rename(columns={col_5 : "VALUE_USD"}, inplace=True)
    return df

# remove unuse columns and sort column order
def remove_columns(df):
    df.drop(columns=['COUNTRY', 
                        'COUNTRY NAME', 
                        'UNIT1', 
                        'UNIT2', 
                        'CURRENT MONTH QUANTITY1', 
                        'CURRENT MONTH QUANTITY2',
                        'CUMULATIVE YEAR TO DATE QUANTITY1',
                        'CUMULATIVE YEAR TO DATE QUANTITY2',
                        'CUMULATIVE YEAR TO DATE VALUE',
                        'Country_Code'], axis=0, inplace=True)
    col_order = ["DATE", "HS_CODE", "ISO", "QUANTITY_PCS", "VALUE_USD"]
    df = df[col_order]
    return df

# to remove rows with empty values
def remove_empty_values_rows(df):
    # remove the rows with empty value in the dataframe
     df_filtered = df[df['VALUE_USD'] != 0]
     return df_filtered
 
# to combine dataframes
def concatenate_dataframes(df1, df2):
    concatenated = pd.concat([df1, df2])
    return concatenated

# create function to loop all csv files within a given range
# and convert into structured format then concat into one 
# collective dataframe
def concat_with_loop(start, end, type, base_df):
    for i in range(start,end):
        df_base_context = pd.read_csv(f"japan_{type} ({i}).csv", header=6)
        df_base_month = pd.read_csv(f"japan_{type} ({i}).csv", on_bad_lines='skip')

        # add date column using defined function
        df_base_context = add_date_column(df_base_month, df_base_context)



        # update iso column using defined function
        df_base_context = update_iso(df_base_context, df_iso, 'COUNTRY')

        # update hs_code column by change the column title and removing symbols
        df_base_context = update_hs_code(df_base_context, "COMMODITY")

        # clean up value by converting to USD and rename column title
        # Note: the parameter need to be the same order as function defined
        # UNIT1 AMOUNT1 UNIT2 AMOUNT2 <<< in this order
        df_base_context = clean_columns(df_base_context, 
                                        "UNIT1",
                                        "UNIT2",
                                        "CURRENT MONTH QUANTITY1",
                                        "CURRENT MONTH QUANTITY2",
                                        "CURRENT MONTH VALUE")

        # remove unused columns and order in the uniform format
        df_base_context = remove_columns(df_base_context)

        # remove the empty value rows
        df_base_context = remove_empty_values_rows(df_base_context)

        # check the context of the existing dataframe
        df_base_context.head(10)

        base_df = concatenate_dataframes(base_df, df_base_context)
        
    base_df.to_csv(f"DB_JP_{type.upper()}_UPDATED_2023_MAR_{date.today().strftime('%Y%m%d')}01.csv", index=False)
        # df_collected


# create empty dataframe with predefined structure
df_base = create_empty_dataframe()

# generate using defined function with parameter
# first argument = start for file name such as japan_export (1).csv >> the starting number is 1
# second argument = end for file name such as japan_export (195).csv is your last file>> the end should be 196 (195 + 1)
# third argument = 'export' or 'import'. be aware the name should be the same. otherwise the system won't find it.
# fourth argument = should define a base dataframe before calling this function 
concat_with_loop(1, 196, 'import', df_base)



C:\Users\hboyz\AppData\Local\Temp\ipykernel_23232\268463701.py:41: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df[col_name] = df[col_name].str.replace(".","")
C:\Users\hboyz\AppData\Local\Temp\ipykernel_23232\268463701.py:41: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df[col_name] = df[col_name].str.replace(".","")
C:\Users\hboyz\AppData\Local\Temp\ipykernel_23232\268463701.py:41: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df[col_name] = df[col_name].str.replace(".","")
C:\Users\hboyz\AppData\Local\Temp\ipykernel

In [100]:
df_base_context["CURRENT MONTH QUANTITY2"] = df_base_context["CURRENT MONTH QUANTITY2"].replace("-",0)
# df_base_context["COMMODITY"] = df_base_context["COMMODITY"].str.replace("-","").astype(int)
df_base_context["CURRENT MONTH QUANTITY2"].head(10)

0      764
1    19933
2     2744
3       12
4    10387
5        0
6      577
7     2973
8      505
9       24
Name: CURRENT MONTH QUANTITY2, dtype: object

In [72]:
df_base_context[df_base_context["ISO"].isna()]

,COMMODITY,COUNTRY,COUNTRY NAME,UNIT1,UNIT2,CURRENT MONTH QUANTITY1,CURRENT MONTH QUANTITY2,CURRENT MONTH VALUE,CUMULATIVE YEAR TO DATE QUANTITY1,CUMULATIVE YEAR TO DATE QUANTITY2,CUMULATIVE YEAR TO DATE VALUE,DATE,Country_Code,ISO


In [48]:
df_template = pd.DataFrame(columns=["DATE", "HS_CODE", "ISO", "COUNTRY_NAME", "AMOUNT_1", "UNIT_1", "AMOUNT_2", "UNIT_2", "VALUE_USD"])
df_template                                    

,DATE,HS_CODE,COUNTRY_CODE,COUNTRY_NAME,AMOUNT_1,UNIT_1,AMOUNT_2,UNIT_2,VALUE_USD


In [42]:
# test year, month
for index, values in df_base_month.iterrows():
    # print(str(df_base_month.loc[index, "Commodity by Country"]).split("/")[0][1:3])
    if str(df_base_month.loc[index, "Commodity by Country"]).split("/")[0][1:3] == "20":
        print(str(df_base_month.loc[index, "Commodity by Country"]).split("/")[0][1:])
        month = str(df_base_month.loc[index, "Commodity by Country"]).split("/")[1]
        if len(month) == 1:
            print('0' + str(month))
        else:
            print(month)

2007
12


In [10]:
df_base_month.head()

,Title,Commodity by Country
0,Export and Import,Export
1,Year and month,'2007/12
2,Country,All
3,UNIT:(1000YEN),NaN


In [5]:
df_export_base_context = pd.read_csv("japan_export (1).csv", header=6)
df_export_base_month = pd.read_csv("japan_export (1).csv", on_bad_lines='skip')

export_month_base = df_export_base_month.loc[1, "Commodity by Country"].split("'")[1] + '/01'

df_export_base_context["DATE"] = export_month_base

for index, value in df_export_base_context.iterrows():
    if df_export_base_context.loc[index,"CURRENT MONTH QUANTITY1"] == "-":
        df_export_base_context.loc[index,"CURRENT MONTH QUANTITY1"] = 0
    if df_export_base_context.loc[index,"CURRENT MONTH QUANTITY2"] == "-":
        df_export_base_context.loc[index,"CURRENT MONTH QUANTITY2"] = 0
    if df_export_base_context.loc[index,"CURRENT MONTH VALUE"] == "-":
        df_export_base_context.loc[index,"CURRENT MONTH VALUE"] = 0
    if df_export_base_context.loc[index,"CUMULATIVE YEAR TO DATE QUANTITY1"] == "-":
        df_export_base_context.loc[index,"CUMULATIVE YEAR TO DATE QUANTITY1"] = 0
    if df_export_base_context.loc[index,"CUMULATIVE YEAR TO DATE QUANTITY2"] == "-":
        df_export_base_context.loc[index,"CUMULATIVE YEAR TO DATE QUANTITY2"] = 0
    if df_export_base_context.loc[index,"CUMULATIVE YEAR TO DATE VALUE"] == "-":
        df_export_base_context.loc[index,"CUMULATIVE YEAR TO DATE VALUE"] = 0
        
    df_export_base_context.loc[index,"COUNTRY"] = str(df_export_base_context.loc[index,"COUNTRY"]).split("'")[1]
    df_export_base_context.loc[index,"COMMODITY"] = str(df_export_base_context.loc[index,"COMMODITY"]).replace(".", "").replace("-","")

for i in range(2,220):
    df_export_context = pd.read_csv(f"japan_export ({i}).csv", header=6)
    df_export_month = pd.read_csv(f"japan_export ({i}).csv", on_bad_lines='skip')

    export_month = df_export_month.loc[1, "Commodity by Country"].split("'")[1] + '/01'

    df_export_context["DATE"] = export_month

    for index, value in df_export_context.iterrows():
        if df_export_context.loc[index,"CURRENT MONTH QUANTITY1"] == "-":
            df_export_context.loc[index,"CURRENT MONTH QUANTITY1"] = 0
        if df_export_context.loc[index,"CURRENT MONTH QUANTITY2"] == "-":
            df_export_context.loc[index,"CURRENT MONTH QUANTITY2"] = 0
        if df_export_context.loc[index,"CURRENT MONTH VALUE"] == "-":
            df_export_context.loc[index,"CURRENT MONTH VALUE"] = 0
        if df_export_context.loc[index,"CUMULATIVE YEAR TO DATE QUANTITY1"] == "-":
            df_export_context.loc[index,"CUMULATIVE YEAR TO DATE QUANTITY1"] = 0
        if df_export_context.loc[index,"CUMULATIVE YEAR TO DATE QUANTITY2"] == "-":
            df_export_context.loc[index,"CUMULATIVE YEAR TO DATE QUANTITY2"] = 0
        if df_export_context.loc[index,"CUMULATIVE YEAR TO DATE VALUE"] == "-":
            df_export_context.loc[index,"CUMULATIVE YEAR TO DATE VALUE"] = 0    
            
        df_export_context.loc[index,"COUNTRY"] = str(df_export_context.loc[index,"COUNTRY"]).split("'")[1]
        df_export_context.loc[index,"COMMODITY"] = str(df_export_context.loc[index,"COMMODITY"]).replace(".", "").replace("-","")

    df_export_base_context = pd.concat([df_export_base_context, df_export_context])

df_export_base_context.to_csv("Japan_Export_2007_2023_Mar_2023052101.csv", index = False)